In [15]:
from functools import lru_cache
import random
import math

# ===========================
# 1. Infinite-deck card model
# ===========================

# Card ranks: 1 = Ace, 2–9, 10 = (10,J,Q,K)
ranks = [1,2,3,4,5,6,7,8,9,10]

# Infinite-deck probabilities (standard 52-card deck)
p_rank = {1:4/52}
for r in range(2,10):
    p_rank[r] = 4/52
p_rank[10] = 16/52

# For Monte Carlo draws
cumulative = []
acc = 0.0
for r in ranks:
    acc += p_rank[r]
    cumulative.append((acc, r))

def draw_rank():
    x = random.random()
    for cutoff, r in cumulative:
        if x <= cutoff:
            return r
    return 10  # fallback

# ===========================
# 2. Hand & dealer logic
# ===========================

def add_card(total, soft, card):
    """
    Return (new_total, new_soft) after adding `card`
    to a hand with current (total, soft).

    - `total` is the current total, counting one Ace as 11 if soft=True.
    - `soft` means we have an Ace counted as 11.
    """
    if card == 1:  # Ace
        # Try to count as 11 if it doesn't bust
        if total + 11 <= 21:
            return total + 11, True
        else:
            new_total = total + 1
            # If we were soft and now bust, convert previous Ace
            if new_total > 21 and soft:
                new_total -= 10
                soft = False
            return new_total, soft
    else:
        val = 10 if card == 10 else card
        total2 = total + val
        soft2 = soft
        # Convert soft Ace if we bust
        if total2 > 21 and soft2:
            total2 -= 10
            soft2 = False
        return total2, soft2

@lru_cache(None)
def dealer_play_from(total, soft):
    """
    Dealer recursion.
    Returns a distribution dict from this state:
        keys in {17,18,19,20,21,22,'bust'}
    Rules:
      - Dealer hits soft 17
      - If final total == 22, we track it separately (for side bet),
        but for the main game it's treated like a bust.
      - If total > 22, it's a normal bust.
    """
    if total >= 17:
        if total <= 21:
            # Dealer hits soft 17
            if total == 17 and soft:
                pass  # continue to hit below
            else:
                return {total: 1.0}
        else:
            # >21
            if total == 22:
                return {22: 1.0}
            else:
                return {'bust': 1.0}

    # Need to hit
    dist = {}
    for r, p in p_rank.items():
        t2, s2 = add_card(total, soft, r)
        for outcome, prob in dealer_play_from(t2, s2).items():
            dist[outcome] = dist.get(outcome, 0.0) + p * prob
    return dist

# Precompute dealer distribution for each upcard (infinite deck)
dealer_dists = {}
for up in ranks:
    dist = {}
    for r, p in p_rank.items():  # hole card
        # Build 2-card starting hand (upcard + hole)
        total, soft = 0, False
        total, soft = add_card(0, False, up)
        total2, soft2 = add_card(total, soft, r)
        for outcome, prob in dealer_play_from(total2, soft2).items():
            dist[outcome] = dist.get(outcome, 0.0) + p * prob
    # Normalize (should be very close to 1)
    s = sum(dist.values())
    for k in dist:
        dist[k] /= s
    dealer_dists[up] = dist

# ===========================
# 3. Player EV: stand vs hit
# ===========================

def is_blackjack(total, soft, num_cards):
    """True if this is a 2-card blackjack (A+10)."""
    return (num_cards == 2 and soft and total == 21)

@lru_cache(None)
def EV_stand(total, soft, num_cards, upcard):
    """
    EV (per unit bet) of standing on `total` vs dealer `upcard`.
    - 2-card blackjack (A+10) always pays 3:2, regardless of dealer.
    - Dealer total 22 is treated as a normal bust for the main wager.
    """
    if total > 21:
        return -1.0

    if is_blackjack(total, soft, num_cards):
        # 2-card blackjack pays 3:2 on the (possibly doubled) bet,
        # and is resolved immediately before dealer draws further.
        return 1.5

    dist = dealer_dists[upcard]
    ev = 0.0
    for outcome, p in dist.items():
        if outcome == 'bust' or outcome == 22:
            ev += p * 1.0  # dealer bust or 22 -> player wins
        else:  # 17-21
            if total > outcome:
                ev += p * 1.0
            elif total < outcome:
                ev += p * (-1.0)
            else:
                ev += p * 0.0  # push
    return ev

@lru_cache(None)
def EV_opt(total, soft, num_cards, upcard):
    """
    Optimal EV from this state with the option to:
      - hit (and then continue optimally), or
      - stand (if num_cards >= 2; you cannot stand on the first card).

    Doubling is not modeled explicitly here; it just multiplies all
    payoffs and does not change card flow, so the optimal hit/stand
    decisions are the same. We'll add doubling suggestions on top.
    """
    if total > 21:
        return -1.0

    # Immediate resolution for 2-card blackjack
    if is_blackjack(total, soft, num_cards):
        return 1.5

    # Stand EV (only allowed once we have at least two cards)
    if num_cards >= 2:
        ev_st = EV_stand(total, soft, num_cards, upcard)
    else:
        ev_st = -1e9  # disallow standing on 1 card

    # Hit EV = expected EV_opt after one card
    ev_hit = 0.0
    for r, p in p_rank.items():
        t2, s2 = add_card(total, soft, r)
        if t2 > 21 and not s2:  # hard bust
            ev_hit += p * (-1.0)
        else:
            ev_hit += p * EV_opt(t2, s2, num_cards + 1, upcard)

    return max(ev_st, ev_hit)

def EV_components(total, soft, num_cards, upcard):
    """Return (EV_stand, EV_hit, EV_opt) for inspection."""
    if num_cards >= 2:
        ev_st = EV_stand(total, soft, num_cards, upcard)
    else:
        ev_st = -1e9
    ev_hit = 0.0
    for r, p in p_rank.items():
        t2, s2 = add_card(total, soft, r)
        if t2 > 21 and not s2:
            ev_hit += p * (-1.0)
        else:
            ev_hit += p * EV_opt(t2, s2, num_cards + 1, upcard)
    return ev_st, ev_hit, max(ev_st, ev_hit)

# ===========================
# 4. Build 2+ card strategy (H/S/D)
# ===========================

actions_hard = {up: {} for up in ranks}
actions_soft = {up: {} for up in ranks}
double_sug_hard = {up: {} for up in ranks}
double_sug_soft = {up: {} for up in ranks}

# Hard totals: 4–21, assuming at least 2 cards
for up in ranks:
    for total in range(4, 22):
        soft = False
        num_cards = 2
        ev_st, ev_hit, ev_opt_val = EV_components(total, soft, num_cards, up)
        actions_hard[up][total] = 'H' if ev_hit > ev_st + 1e-9 else 'S'
        # Suggest double if it's a hitting hand with positive EV
        double_sug_hard[up][total] = (ev_hit > ev_st + 1e-9 and ev_opt_val > 0)

# Soft totals: 13–21 (A+2 up to A+10), at least 2 cards
for up in ranks:
    for total in range(13, 22):
        soft = True
        num_cards = 2
        ev_st, ev_hit, ev_opt_val = EV_components(total, soft, num_cards, up)
        actions_soft[up][total] = 'H' if ev_hit > ev_st + 1e-9 else 'S'
        double_sug_soft[up][total] = (ev_hit > ev_st + 1e-9 and ev_opt_val > 0)

# ===========================
# 5. One-card strategy (H vs D)
# ===========================

def one_card_EV(card, upcard):
    """EV of playing optimally starting from a single card."""
    total, soft = add_card(0, False, card)
    return EV_opt(total, soft, 1, upcard)

# For 1-card states we only need H vs D (you cannot stand)
one_card_action = {card: {} for card in ranks}  # H or D

for card in ranks:
    for up in ranks:
        ev = one_card_EV(card, up)
        # Risk-neutral rule: double when EV > 0, otherwise just hit.
        one_card_action[card][up] = 'D' if ev > 0 else 'H'

# ===========================
# 6. Pretty-print strategy tables
# ===========================

def build_table_hard():
    cols = list(range(2, 11)) + [1]
    lines = []
    header = "Total | " + " ".join(f"{c:>2}" if c != 1 else " A" for c in cols)
    lines.append(header)
    lines.append("-" * len(header))
    for total in range(4, 22):
        row = f"{total:>5} | "
        for up in cols:
            act = actions_hard[up][total]
            if double_sug_hard[up][total]:
                act = 'D'
            row += f" {act} "
        lines.append(row)
    return "\n".join(lines)

def build_table_soft():
    cols = list(range(2, 11)) + [1]
    lines = []
    header = "Soft | " + " ".join(f"{c:>2}" if c != 1 else " A" for c in cols)
    lines.append(header)
    lines.append("-" * len(header))
    for total in range(13, 22):
        row = f"{total:>4} | "
        for up in cols:
            act = actions_soft[up][total]
            if double_sug_soft[up][total]:
                act = 'D'
            row += f" {act} "
        lines.append(row)
    return "\n".join(lines)

def build_table_one_card():
    cols = list(range(2, 11)) + [1]
    lines = []
    header = "Card | " + " ".join(f"{c:>2}" if c != 1 else " A" for c in cols)
    lines.append(header)
    lines.append("-" * len(header))
    for card in ranks:
        label = 'A' if card == 1 else str(card)
        row = f"{label:>4} | "
        for up in cols:
            act = one_card_action[card][up]  # H or D
            row += f" {act} "
        lines.append(row)
    return "\n".join(lines)

# ===========================
# 7. Monte Carlo verification
# ===========================

def simulate_hand(num_rounds=100_000, seed=12345):
    """
    Monte Carlo simulation of full hands using the derived strategy.
    Returns estimated EV per initial unit bet.
    """
    random.seed(seed)
    profit_sum = 0.0

    for _ in range(num_rounds):
        # Initial bet
        total_bet = 1.0

        # Dealer cards
        up = draw_rank()
        hole = draw_rank()
        d_total, d_soft = add_card(0, False, up)
        d_total, d_soft = add_card(d_total, d_soft, hole)

        # Player first card
        first = draw_rank()
        p_total, p_soft = add_card(0, False, first)
        num_cards = 1

        # Player turn
        busted = False
        blackjack_paid = False

        while True:
            # Check for 2-card blackjack
            if is_blackjack(p_total, p_soft, num_cards):
                profit_sum += 1.5 * total_bet
                blackjack_paid = True
                break

            if p_total > 21:
                profit_sum -= total_bet
                busted = True
                break

            if num_cards == 1:
                # One-card decision: H or D
                act = one_card_action[first][up]
                if act == 'D':
                    total_bet *= 2
                # Always hit on 1 card (standing not allowed)
                card = draw_rank()
                p_total, p_soft = add_card(p_total, p_soft, card)
                num_cards += 1
                continue

            # num_cards >= 2 and not blackjack/bust
            # Choose action from the hard/soft tables
            if p_soft and p_total >= 13:
                table = actions_soft
                double_table = double_sug_soft
            else:
                table = actions_hard
                double_table = double_sug_hard

            act = table[up].get(p_total, 'S')
            if double_table[up].get(p_total, False):
                act = 'D'

            if act == 'H':
                card = draw_rank()
                p_total, p_soft = add_card(p_total, p_soft, card)
                num_cards += 1
            elif act == 'D':
                total_bet *= 2
                card = draw_rank()
                p_total, p_soft = add_card(p_total, p_soft, card)
                num_cards += 1
            else:  # 'S'
                break

        if busted or blackjack_paid:
            continue

        # Dealer plays out hand
        while True:
            if d_total >= 17:
                if d_total > 21:
                    # 22 is treated as bust for main bet
                    profit_sum += total_bet
                else:
                    # Compare totals 17–21
                    if p_total > d_total:
                        profit_sum += total_bet
                    elif p_total < d_total:
                        profit_sum -= total_bet
                    else:
                        pass  # push
                break
            # Dealer hits
            card = draw_rank()
            d_total, d_soft = add_card(d_total, d_soft, card)

    return profit_sum / num_rounds

if __name__ == "__main__":
    print("=== HARD TOTALS (H/S/D) ===")
    print(build_table_hard())
    print()
    print("=== SOFT TOTALS (H/S/D) ===")
    print(build_table_soft())
    print()
    print("=== ONE-CARD STRATEGY (H = hit, D = double) ===")
    print(build_table_one_card())
    print()
    ev_est = simulate_hand(num_rounds=1000000)
    print(f"Monte Carlo estimated EV per initial unit: {ev_est:.5f}")


=== HARD TOTALS (H/S/D) ===
Total |  2  3  4  5  6  7  8  9 10  A
-------------------------------------
    4 |  H  H  H  H  D  H  H  H  H  H 
    5 |  H  H  H  H  D  H  H  H  H  H 
    6 |  H  H  H  H  D  H  H  H  H  H 
    7 |  H  H  H  H  D  H  H  H  H  H 
    8 |  H  D  D  D  D  D  H  H  H  H 
    9 |  D  D  D  D  D  D  D  H  H  H 
   10 |  D  D  D  D  D  D  D  D  H  H 
   11 |  D  D  D  D  D  D  D  D  D  H 
   12 |  H  H  S  S  S  H  H  H  H  H 
   13 |  S  S  S  S  S  H  H  H  H  H 
   14 |  S  S  S  S  S  H  H  H  H  H 
   15 |  S  S  S  S  S  H  H  H  H  H 
   16 |  S  S  S  S  S  H  H  H  H  H 
   17 |  S  S  S  S  S  S  S  S  S  S 
   18 |  S  S  S  S  S  S  S  S  S  S 
   19 |  S  S  S  S  S  S  S  S  S  S 
   20 |  S  S  S  S  S  S  S  S  S  S 
   21 |  S  S  S  S  S  S  S  S  S  S 

=== SOFT TOTALS (H/S/D) ===
Soft |  2  3  4  5  6  7  8  9 10  A
------------------------------------
  13 |  D  D  D  D  D  D  D  H  H  H 
  14 |  D  D  D  D  D  D  D  H  H  H 
  15 |  D  D  D